In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [8]:
import logging
import pandas as pd

from general_functions.call_api_with_account_id import send_to_innkeepr_api_paginated
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url

In [ ]:
customer = "LILLYDOO"
start_date = "2025-02-03"
end_date = "2025-06-06"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]
path_to_save_data = "/Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/SprintStories/PRD-2461-Lillydoo-smart-bidding/data/"
save_conversions=f"{path_to_save_data}conversions_{customer}_{start_date}_{end_date}.parquet"

In [12]:
try:
    conversions = pd.read_parquet(save_conversions)
except FileNotFoundError:
    print(f"Query conversions")
    content = {'created': {'$gte': start_date, '$lte': end_date}, 'name': 'checkout_completed'}
    resp = send_to_innkeepr_api_paginated(
        f"{url}/conversions/query",
        account_id,
        content,
        logging
    )
    conversions = pd.json_normalize(resp)
    conversions.to_parquet(save_conversions)
conversions

In [13]:
conversions["created"].min(), conversions["created"].max()

In [14]:
conversions["name"].value_counts()

In [15]:
sessions_resp = send_to_innkeepr_api_paginated(
        f"{url}/sessions/query",
        account_id,
        {"sessionId":conversions["sessionId"].dropna().unique().tolist()},
        logging
    )
sessions = pd.json_normalize(sessions_resp)
sessions

In [16]:
sessions_with_gclid = sessions[sessions["campaign.gclid"].isnull()==False]
sessions_with_gclid.shape

In [17]:
sessions_with_gclid_italy = sessions_with_gclid[sessions_with_gclid["referrer"].str.contains("com/it")]
sessions_with_gclid_italy.shape

In [20]:
sessions_with_gclid_italy = sessions_with_gclid[sessions_with_gclid["referrer"].str.contains("it/")]
print(sessions_with_gclid_italy.shape)
sessions_with_gclid_italy

# Match with adAccountId

In [22]:
signals_resp = send_to_innkeepr_api_paginated(
        f"{url}/signals/query",
        account_id,
        {"externalId":sessions_with_gclid_italy["campaign.gclid"].dropna().unique().tolist()},
        logging
    )
signals = pd.json_normalize(signals_resp)
signals

In [25]:
signals_filtered = signals[signals["relates_to.adGroupAd.resourceName"].str.contains(adaccountid)]
signals_filtered

In [24]:
adaccountid = "8234087217"
for col in signals.columns:
    temp = signals[col].str.contains(adaccountid)
    if temp.sum()>0:
        print(col, temp.sum())